In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import geopandas as gpd
import math

from api_config import geocode_postcode, fetch_isochrone, osrm_route, PostcodeNotFound
from geo_config import isochrone_to_gdf, teams_to_gdf, filter_teams_by_minutes, list_business_units, apply_team_filters

In [ ]:
# Ground Control palette
humannature = {
    'GC Dark Green': '#294238',
    'GC Light Green': '#b2d235',
    'GC Mid Green': '#50b748',
    'GC Orange': '#f57821',
    'GC Light Grey': '#e6ebe3'
}

In [ ]:
def _apply_styles(root: tk.Tk):
    root.configure(bg=humannature["GC Light Grey"])
    style = ttk.Style(root)
    try:
        style.theme_use("clam")
    except Exception:
        pass

    style.configure("TFrame", background=humannature["GC Light Grey"])
    style.configure("TLabel", background=humannature["GC Light Grey"], foreground=humannature["GC Dark Green"])
    style.configure("Header.TLabel", font=("Segoe UI", 11, "bold"), foreground=humannature["GC Dark Green"])
    style.configure("TButton", padding=8, foreground="white", background=humannature["GC Mid Green"])
    style.map("TButton", background=[("active", humannature["GC Light Green"])])

    # Go! button
    style.configure("Go.TButton", padding=12, font=("Segoe UI", 11, "bold"),
                    foreground="white", background=humannature["GC Orange"])
    style.map("Go.TButton", background=[("active", "#ff8f3b")])


def _minutes_from_tab(notebook: ttk.Notebook) -> int:
    # Map active tab index to minutes contours
    idx = notebook.index(notebook.select())
    return [15, 30, 45, 60][idx]


def _haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0088
    import math
    p1 = math.radians(lat1); p2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
    return 2*R*math.asin(math.sqrt(a))


class ResourceFinderUI:
    def __init__(self, root: tk.Tk, fieldteams: pd.DataFrame):
        self.root = root
        self.root.title("Field Team Resource Finder")
        self.df = fieldteams

        # state
        self.site_lon = None
        self.site_lat = None
        self.iso_gdf = None

        # tk vars
        self.pc_var = tk.StringVar(value="")
        self.bu_var = tk.StringVar(value="(Any)")
        self.sla_var = tk.BooleanVar(value=False)  # coming soon

        _apply_styles(self.root)
        self._build()

    def _build(self):
        # Controls rows
        top = ttk.Frame(self.root); top.grid(row=0, column=0, sticky="ew", padx=12, pady=8)
        top.grid_columnconfigure(0, weight=1)
        top.grid_columnconfigure(1, weight=1)
        top.grid_columnconfigure(2, weight=1)
        top.grid_columnconfigure(3, weight=0)

        ttk.Label(top, text="Postcode:").grid(row=0, column=0, sticky="w")
        ttk.Entry(top, textvariable=self.pc_var, width=16).grid(row=1, column=0, sticky="w", padx=(0,8))

        ttk.Label(top, text="Business Unit:").grid(row=0, column=1, sticky="w")
        self.bu_combo = ttk.Combobox(top, textvariable=self.bu_var, width=28, state="readonly")

        # Initial drop-down options from full dataset; refined after first Go
        self.bu_combo["values"] = list_business_units(self.df) if "BusinessUnit" in self.df.columns else ["(Any)"]
        self.bu_combo.set("(Any)")
        self.bu_combo.grid(row=1, column=1, sticky="w", padx=(0,8))

        ttk.Label(top, text="Include teams not meeting SLAs (coming soon)").grid(row=0, column=2, sticky="w")
        ttk.Checkbutton(top, variable=self.sla_var).grid(row=1, column=2, sticky="w", padx=(0,8))

        ttk.Button(top, text="Go!", style="Go.TButton", command=self.on_go).grid(row=0, column=3, rowspan=2, sticky="e")

        # Minutes contour tabs
        self.tabs = ttk.Notebook(self.root)
        self.tabs.grid(row=1, column=0, sticky="ew", padx=12)
        for label in ("Within 15 Minutes", "Within 30 Minutes", "Within 45 Minutes", "Within 60 Minutes"):
            frame = ttk.Frame(self.tabs)
            self.tabs.add(frame, text=label)

        # Split area: table and map placeholder
        split = ttk.Frame(self.root); split.grid(row=2, column=0, sticky="nsew", padx=12, pady=(6,12))
        self.root.grid_rowconfigure(2, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        split.grid_columnconfigure(0, weight=1)
        split.grid_columnconfigure(1, weight=1)
        split.grid_rowconfigure(0, weight=1)

        # Table
        left = ttk.Frame(split); left.grid(row=0, column=0, sticky="nsew", padx=(0,6))
        cols = ("Team Name", "Contact #", "Distance")
        self.tree = ttk.Treeview(left, columns=cols, show="headings", height=18)
        for c, w in zip(cols, (220, 140, 120)):
            self.tree.heading(c, text=c)
            self.tree.column(c, width=w, anchor="w")
        self.tree.grid(row=0, column=0, sticky="nsew")
        sb = ttk.Scrollbar(left, orient="vertical", command=self.tree.yview)
        self.tree.configure(yscroll=sb.set)
        sb.grid(row=0, column=1, sticky="ns")
        left.grid_rowconfigure(0, weight=1)
        left.grid_columnconfigure(0, weight=1)

        # Map placeholder
        right = ttk.Frame(split); right.grid(row=0, column=1, sticky="nsew", padx=(6,0))
        self.map_canvas = tk.Canvas(right, bg="#d9d9d9", highlightthickness=0)
        self.map_canvas.grid(row=0, column=0, sticky="nsew")
        right.grid_rowconfigure(0, weight=1)
        right.grid_columnconfigure(0, weight=1)
        self._draw_placeholder()

    def _draw_placeholder(self):
        c = self.map_canvas
        c.delete("all")
        w = c.winfo_width() or 600
        h = c.winfo_height() or 400
        pad = 20
        c.create_rectangle(pad, pad, w-pad, h-pad, fill="#cfcfcf", outline="")
        c.create_text(w/2, h/2, text="Map preview\n(coming soon)", font=("Segoe UI", 12), fill="#555555")

    def _populate_table(self, df: pd.DataFrame):
        self.tree.delete(*self.tree.get_children())
        for _, r in df.iterrows():
            self.tree.insert("", "end", values=(
                r.get("Contractor", ""),
                r.get("Contact", "—"),
                r.get("Distance", ""),
            ))

    def on_go(self):
        pc = self.pc_var.get().strip()
        if not pc:
            messagebox.showinfo("Input needed", "Please enter a postcode.")
            return

        try:
            self.site_lon, self.site_lat = geocode_postcode(pc)
        except PostcodeNotFound as e:
            messagebox.showerror("Invalid postcode", str(e))
            return
        except Exception as e:
            messagebox.showerror("Error", f"Failed to geocode: {e}")
            return

        # One isochrone call (all contours); then filter locally by selected tab
        try:
            iso_gj = fetch_isochrone(self.site_lon, self.site_lat)
            self.iso_gdf = isochrone_to_gdf(iso_gj)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to fetch isochrones: {e}")
            return

        minutes = _minutes_from_tab(self.tabs)
        teams_gdf = teams_to_gdf(self.df)
        within = filter_teams_by_minutes(teams_gdf, self.iso_gdf, minutes=minutes)

        # BU filter
        bu_choice = self.bu_var.get()
        # Internal/Contractor filter omitted to match wireframe; can be added later
        filtered = apply_team_filters(within, business_unit=bu_choice, internal_flag=None)

        # Update BU list based on the current result set
        self.bu_combo["values"] = list_business_units(filtered)
        if bu_choice not in self.bu_combo["values"]:
            self.bu_combo.set("(Any)")

        # Keep OSRM calls modest (preselect nearest by air distance)
        if filtered.empty:
            self._populate_table(pd.DataFrame())
            messagebox.showinfo("No teams", "No teams found in the selected contour and filters.")
            return

        # Preselect top N by air distance to reduce OSRM calls
        N = 20
        cand = filtered.copy()
        cand["air_km"] = [
            _haversine_km(self.site_lat, self.site_lon, float(r["Latitude"]), float(r["Longitude"]))
            for _, r in cand.iterrows()
        ]
        cand = cand.sort_values("air_km").head(N)

        # Call OSRM for candidates
        rows = []
        for _, r in cand.iterrows():
            try:
                res = osrm_route(
                    start_lon=float(r["Longitude"]),
                    start_lat=float(r["Latitude"]),
                    dest_lon=self.site_lon,
                    dest_lat=self.site_lat,
                )
                rows.append({
                    "Contractor": r.get("Contractor", ""),
                    "Contact": "—",  # placeholder until contact data is available
                    "Distance": f"{res['distance_km']:.1f} km",
                    # You might also calculate and display time / CO2 in future columns
                })
            except Exception as e:
                messagebox.showerror("Routing error", f"Failed to route {r.get('Contractor','(unknown)')}: {e}")

        out = pd.DataFrame(rows).sort_values("Distance").reset_index(drop=True)
        self._populate_table(out)


# Launch function 
def launch_gui(fieldteams: pd.DataFrame):
    root = tk.Tk()
    ui = ResourceFinderUI(root, fieldteams)
    root.geometry("1000x640")
    root.minsize(900, 560)
    # re-draw placeholder when the canvas resizes
    def _on_resize(event):
        ui._draw_placeholder()
    ui.map_canvas.bind("<Configure>", _on_resize)
    root.mainloop()